In [ ]:
import json
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
os.chdir("/gdrive/My Drive/SemEval2024_Task3")
os.listdir()

['Subtask_1_trial.json',
 'Subtask_2_trial.json',
 'video',
 'text',
 'video_with_audio',
 'README.md',
 'span_extraction_config.json',
 'EmotionalClassifier_bert-base-uncased-emotion.pt',
 'EmotionalTokenizer_bert-base-uncased-emotion.pt',
 'EmotionalTokenizer',
 'Emo_classifier_and_span.pt',
 'validation_prediction_data',
 'SemEval-Task3.ipynb']

In [ ]:
with open('text/Subtask_1_train.json') as f:
    data = json.load(f)

In [ ]:
data[0]

{'conversation_ID': 1,
 'conversation': [{'utterance_ID': 1,
   'text': 'Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .',
   'speaker': 'Chandler',
   'emotion': 'neutral'},
  {'utterance_ID': 2,
   'text': 'Oh , yeah . Had that dream .',
   'speaker': 'All',
   'emotion': 'neutral'},
  {'utterance_ID': 3,
   'text': 'Then I look down , and I realize there is a phone ... there .',
   'speaker': 'Chandler',
   'emotion': 'surprise'},
  {'utterance_ID': 4,
   'text': 'Instead of ... ?',
   'speaker': 'Joey',
   'emotion': 'surprise'},
  {'utterance_ID': 5,
   'text': 'That is right .',
   'speaker': 'Chandler',
   'emotion': 'anger'},
  {'utterance_ID': 6,
   'text': 'Never had that dream .',
   'speaker': 'Joey',
   'emotion': 'neutral'},
  {'utterance_ID': 7,
   'text': 'No .',
   'speaker': 'Phoebe',
   'emotion': 'neutral'},
  {'utterance_ID': 8,
   'text': 'All of a sudden , the phone starts to ring .',
   's

In [ ]:
from sgnlp.models.emotion_entailment import (
    RecconEmotionEntailmentConfig,
    RecconEmotionEntailmentTokenizer,
    RecconEmotionEntailmentModel,
    RecconEmotionEntailmentPreprocessor,
    RecconEmotionEntailmentPostprocessor,
)

In [ ]:
config = RecconEmotionEntailmentConfig.from_pretrained(
    "https://storage.googleapis.com/sgnlp-models/models/reccon_emotion_entailment/config.json"
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:604: UserWarning: Using `from_pretrained` with the url of a file (here https://storage.googleapis.com/sgnlp-models/models/reccon_emotion_entailment/config.json) is deprecated and won't be possible anymore in v5 of Transformers. You should host your file on the Hub (hf.co) instead and use the repository ID. Note that this is not compatible with the caching system (your file will be downloaded at each execution) or multiple processes (each process will download the file in a different temporary file).
  warnings.warn(


(…)ls/reccon_emotion_entailment/config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

In [ ]:
tokenizer = RecconEmotionEntailmentTokenizer.from_pretrained("roberta-base")


NameError: ignored

In [ ]:
model = RecconEmotionEntailmentModel.from_pretrained(
    "https://storage.googleapis.com/sgnlp-models/models/reccon_emotion_entailment/pytorch_model.bin",
    config=config,
)


/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:604: UserWarning: Using `from_pretrained` with the url of a file (here https://storage.googleapis.com/sgnlp-models/models/reccon_emotion_entailment/pytorch_model.bin) is deprecated and won't be possible anymore in v5 of Transformers. You should host your file on the Hub (hf.co) instead and use the repository ID. Note that this is not compatible with the caching system (your file will be downloaded at each execution) or multiple processes (each process will download the file in a different temporary file).
  warnings.warn(


(…)con_emotion_entailment/pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [ ]:
preprocessor = RecconEmotionEntailmentPreprocessor(tokenizer)
postprocessor = RecconEmotionEntailmentPostprocessor()

In [ ]:
input_batch = {
    "emotion": ["happiness", "happiness", "happiness", "happiness"],
    "target_utterance": [
        "Thank you very much .",
        "Thank you very much .",
        "Thank you very much .",
        "Thank you very much .",
    ],
    "evidence_utterance": [
        "It's very thoughtful of you to invite me to your wedding .",
        "How can I forget my old friend ?",
        "My best wishes to you and the bride !",
        "Thank you very much .",
    ],
    "conversation_history": [
        "It's very thoughtful of you to invite me to your wedding . How can I forget my old friend ? My best wishes to you and the bride ! Thank you very much .",
        "It's very thoughtful of you to invite me to your wedding . How can I forget my old friend ? My best wishes to you and the bride ! Thank you very much .",
        "It's very thoughtful of you to invite me to your wedding . How can I forget my old friend ? My best wishes to you and the bride ! Thank you very much .",
        "It's very thoughtful of you to invite me to your wedding . How can I forget my old friend ? My best wishes to you and the bride ! Thank you very much .",
    ],
}

In [ ]:
tensor_dict = preprocessor(input_batch)


In [ ]:
tokenizer.decode(tensor_dict['input_ids'][0])

"<s> happiness <SEP> Thank you very much. <SEP> It's very thoughtful of you to invite me to your wedding. <SEP> It's very thoughtful of you to invite me to your wedding. How can I forget my old friend? My best wishes to you and the bride! Thank you very much.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [ ]:
tensor_dict

{'input_ids': tensor([[    0, 11098, 28696,  ...,     1,     1,     1],
        [    0, 11098, 28696,  ...,     1,     1,     1],
        [    0, 11098, 28696,  ...,     1,     1,     1],
        [    0, 11098, 28696,  ...,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
raw_output = model(**tensor_dict)


In [ ]:
raw_output

SequenceClassifierOutput(loss=None, logits=tensor([[-0.9492,  0.9033],
        [ 1.0538, -1.1472],
        [-0.6174,  0.5001],
        [ 1.0831, -1.1176]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
output = postprocessor(raw_output)
output

[1, 0, 1, 0]

In [ ]:
from sgnlp.models.span_extraction import (
    RecconSpanExtractionConfig,
    RecconSpanExtractionModel,
    RecconSpanExtractionTokenizer,
    RecconSpanExtractionPreprocessor,
    RecconSpanExtractionPostprocessor,
)

config = RecconSpanExtractionConfig.from_pretrained(
"span_extraction_config.json"
)
tokenizer = RecconSpanExtractionTokenizer.from_pretrained(
    "mrm8488/spanbert-finetuned-squadv2"
)
model = RecconSpanExtractionModel.from_pretrained(
    "aisingapore/SPANBert",
    config=config,
)
preprocessor = RecconSpanExtractionPreprocessor(tokenizer)
postprocessor = RecconSpanExtractionPostprocessor()

input_batch = {
    "emotion": ["surprise", "surprise"],
    "target_utterance": [
        "Hi George ! It's good to see you !",
        "Hi George ! It's good to see you !",
    ],
    "evidence_utterance": [
        "Linda ? Is that you ? I haven't seen you in ages !",
        "Hi George ! It's good to see you !",
    ],
    "conversation_history": [
        "Linda ? Is that you ? I haven't seen you in ages ! Hi George ! It's good to see you !",
        "Linda ? Is that you ? I haven't seen you in ages ! Hi George ! It's good to see you !",
    ],
}

tensor_dict, evidences, examples, features = preprocessor(input_batch)
raw_output = model(**tensor_dict)
context, evidence_span, probability = postprocessor(raw_output, evidences, examples, features)

print(context)
# [['Linda ? Is that you ? ', "I haven't seen you in ages !"], ['Hi George ! ', "It's good to see you !"]]
print(evidence_span)
# [[0, 1], [0, 1]]
print(probability)
# [[-1, 0.943615029866203], [-1, 0.8712913786944898]]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RecconSpanExtractionTokenizer'.


pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

RuntimeError: ignored

In [ ]:
from sgnlp.models.span_extraction import (
    RecconSpanExtractionConfig,
    RecconSpanExtractionModel,
    RecconSpanExtractionTokenizer,
    RecconSpanExtractionPreprocessor,
    RecconSpanExtractionPostprocessor,
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("mrm8488/spanbert-finetuned-squadv2")
model = AutoModelForQuestionAnswering.from_pretrained("aisingapore/SPANBert")

In [ ]:
input_batch = {
    "emotion": ["surprise", "surprise"],
    "target_utterance": [
        "Hi George ! It's good to see you !",
        "Hi George ! It's good to see you !",
    ],
    "evidence_utterance": [
        "Linda ? Is that you ? I haven't seen you in ages !",
        "Hi George ! It's good to see you !",
    ],
    "conversation_history": [
        "Linda ? Is that you ? I haven't seen you in ages ! Hi George ! It's good to see you lol !",
        "Linda ? Is that you ? I haven't seen you in ages ! Hi George ! It's good to see you !",
    ],
}

In [ ]:
from sgnlp.models.span_extraction import (
    RecconSpanExtractionConfig,
    RecconSpanExtractionModel,
    RecconSpanExtractionTokenizer,
    RecconSpanExtractionPreprocessor,
    RecconSpanExtractionPostprocessor,
)

config = RecconSpanExtractionConfig.from_pretrained(
"span_extraction_config.json"
)
tokenizer = RecconSpanExtractionTokenizer.from_pretrained(
    "mrm8488/spanbert-finetuned-squadv2")

preprocessor = RecconSpanExtractionPreprocessor(tokenizer)
postprocessor = RecconSpanExtractionPostprocessor()


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RecconSpanExtractionTokenizer'.


In [ ]:
tensor_dict, evidences, examples, features = preprocessor(input_batch)

add example index and unique id: 100%|██████████| 2/2 [00:00<00:00, 15307.68it/s]


In [ ]:
raw_output = model(**tensor_dict)


In [ ]:
context, evidence_span, probability = postprocessor(raw_output, evidences, examples, features)
print(context)
# [['Linda ? Is that you ? ', "I haven't seen you in ages !"], ['Hi George ! ', "It's good to see you !"]]
print(evidence_span)
# [[0, 1], [0, 1]]
print(probability)
# [[-1, 0.943615029866203], [-1, 0.8712913786944898]]

[['Linda ? Is that you ? ', "I haven't seen you in ages !"], ['Hi George ! ', "It's good to see you !"]]
[[0, 1], [0, 1]]
[[-1, 0.9397247001178719], [-1, 0.8712917171246147]]


In [ ]:
model(inpout['input_ids'], )

add example index and unique id: 100%|██████████| 2/2 [00:00<00:00, 13934.56it/s]


TypeError: ignored

In [ ]:
tokenizer.decode(inpout['input_ids'][1])

"[CLS] The target utterance is Hi George! It's good to see you! The evidence utterance is Hi George! It's good to see you! What is the causal span from context that is relevant to the target utterance's emotion surprise? [SEP] Linda? Is that you? I haven't seen you in ages! Hi George! It's good to see you! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("Nakul24/Spanbert-emotion-extraction")
model = AutoModelForQuestionAnswering.from_pretrained("Nakul24/Spanbert-emotion-extraction")

tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

In [ ]:
text_inp = tokenizer(input_batch['target_utterance'], input_batch['evidence_utterance'], input_batch['conversation_history'], padding=True)

In [ ]:
tokenizer.decode(tokenizer(input_batch['target_utterance'], [x[0] +"[SEP]" + x[1] for x in zip(input_batch['evidence_utterance'], input_batch['conversation_history'])])['input_ids'][0])

"[CLS] hi george! it's good to see you! [SEP] linda? is that you? i haven't seen you in ages! [SEP] linda? is that you? i haven't seen you in ages! hi george! it's good to see you lol! [SEP]"

In [ ]:
tokenizer(input_batch['target_utterance'], input_batch['conversation_history'])['input_ids'].shape

AttributeError: ignored

In [ ]:
text_inp

{'input_ids': [[101, 20844, 176, 25690, 2176, 106, 1122, 112, 188, 1363, 1106, 1267, 1128, 106, 102, 181, 20344, 136, 1110, 1115, 1128, 136, 178, 3983, 112, 189, 1562, 1128, 1107, 6776, 106, 102], [101, 20844, 176, 25690, 2176, 106, 1122, 112, 188, 1363, 1106, 1267, 1128, 106, 102, 20844, 176, 25690, 2176, 106, 1122, 112, 188, 1363, 1106, 1267, 1128, 106, 102, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]], 'labels': [[101, 181, 20344, 136, 1110, 1115, 1128, 136, 178, 3983, 112, 189, 1562, 1128, 1107, 6776, 106, 20844, 176, 25690, 2176, 106, 1122, 112, 188, 1363, 1106, 1267, 1128, 25338, 1233, 106,

In [ ]:
out = model(torch.tensor(text_inp['input_ids']), # The tokens representing our input text.
                token_type_ids=torch.tensor(text_inp['token_type_ids']), attention_mask=torch.tensor(text_inp['attention_mask']))

In [ ]:
print (torch.argmax(out['start_logits'], axis=1))
torch.argmax(out['end_logits'], axis=1)


tensor([2, 8])


tensor([30, 13])

In [ ]:
tokenizer.decode(text_inp['input_ids'][0][2:31])

"george! it's good to see you! [SEP] linda? is that you? i haven't seen you in ages!"

In [ ]:
out['start_logits'].shape

torch.Size([2, 47])

In [ ]:
len(text_inp['input_ids'][0])

47

In [ ]:
def decode(conversation):
    emotions = [i['emotion'] for i in conversation['conversation']]
    texts = [i['text'] for i in conversation['conversation']]
    history = ['['+i['speaker']+']'+i['text'] for i in conversation['conversation']]
    history = "".join(history)
    emotion = []
    target_utterence = []
    evidence_utterence = []
    conversation_history = []
    labels = []
    for i in range(0, len(emotions)):
        if (emotions[i] != 'neutral'):
            emotion = emotion+([emotions[i]]*len(texts))
            target_utterence = target_utterence+([texts[i]]*len(texts))
            evidence_utterence = evidence_utterence+(texts)
            conversation_history = conversation_history+([history]*len(texts))
    batch = {'emotion':emotion,"target_utterance":target_utterence,
             "evidence_utterance":evidence_utterence,"conversation_history":conversation_history}
    return batch



In [ ]:
decode(data[0])

{'emotion': ['surprise',
  'surprise',
  'surprise',
  'surprise',
  'surprise',
  'surprise',
  'surprise',
  'surprise',
  'surprise',
  'surprise',
  'surprise',
  'surprise',
  'surprise',
  'surprise',
  'surprise',
  'surprise',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger'],
 'target_utterance': ['Then I look down , and I realize there is a phone ... there .',
  'Then I look down , and I realize there is a phone ... there .',
  'Then I look down , and I realize there is a phone ... there .',
  'Then I look down , and I realize there is a phone ... there .',
  'Then I look down , and I realize there is a phone ... there .',
  'Then I look down , and I realize there is a phone ... there .',
  'Then I look down , and I realize there is a phone ... there .',
  'Then I look down , and I realize there is a phone ... there .',
  'Instead of ... ?',
  'Instead of ... ?',
  'Instead of ... ?',
  'Instead of ... ?',
  'Instead of ... ?',
  'Instead

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Load pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Add new tokens
new_tokens = ["[NEW_TOKEN1]", "[NEW_TOKEN2]"]
tokenizer.add_tokens(new_tokens)

# Resize the model's embedding layer to accommodate the new tokens
model = AutoModel.from_pretrained("bert-base-uncased")
model.resize_token_embeddings(len(tokenizer))

# Example: Tokenize a sentence with the new tokens
text = "This is an example sentence with [NEW_TOKEN1]."
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
print(tokens)

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# Load the pretrained model and tokenizer
model_checkpoint = "Nakul24/Spanbert-emotion-extraction"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

# Add new tokens to the tokenizer
new_tokens = {'additional_special_tokens': ["[SP1]", "[SP2]", "[SP3]", "[SP4]", "[SP5]", "[SP6]", "[SP7]", "[SP8]", "[SEP]"]}
tokenizer.add_special_tokens(new_tokens)

# Resize the model's embedding layer to accommodate the new tokens
# model.resize_token_embeddings(len(tokenizer))

# Use the modified tokenizer and model
# nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

# Example usage
context = ["[SP1] This is an example sentence.", "[SP2] This is another example sentence."]
question = ["What is an example?", "This is an example sentence."]

# Tokenize with the modified tokenizer
inputs = tokenizer(context, question, return_tensors="pt", padding=True)

print (inputs)

# Forward pass with the modified model
# outputs = model(**inputs)

# Extract answers
# start_logits, end_logits = outputs.start_logits, outputs.end_logits
# start_index = torch.argmax(start_logits)
# end_index = torch.argmax(end_logits) + 1
# answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index])

# print("Answer:", answer)


{'input_ids': tensor([[  101, 29001,  1142,  1110,  1126,  1859,  5650,   119,   102,  1184,
          1110,  1126,  1859,   136,   102,     0],
        [  101, 29003,  1142,  1110,  1330,  1859,  5650,   119,   102,  1142,
          1110,  1126,  1859,  5650,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
tokenizer.decode(inputs['input_ids'][1])

'[CLS] [SP2] this is another example sentence. [SEP] this is an example sentence. [SEP]'

In [ ]:
tokenizer.vocab['[SEP]']

102

In [ ]:
from transformers import AutoTokenizer

# Assume you have a pretrained model, e.g., DistilBERT
pretrained_model = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

# Your input text
input_text = "This is an example text. The answer is in this span."

# Tokenize the text
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(input_text)))

# Find the span indices in the tokenized sequence
start_span = input_text.find("answer")
end_span = start_span + len("answer") - 1  # Assuming inclusive indexing

# Find the corresponding token indices
start_token = len(tokenizer.tokenize(tokenizer.decode(tokenizer.encode(input_text[:start_span]))))
end_token = start_token + len(tokenizer.tokenize(tokenizer.decode(tokenizer.encode(input_text[start_span:end_span + 1])))) - 1

print("Token Indices:", start_token, end_token)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Token Indices: 9 11


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForQuestionAnswering, BertModel, RobertaModel, AutoModelForSequenceClassification, AutoTokenizer

class SpanClassificationModel(nn.Module):
    def __init__(self, span_model_checkpoint, classification_model_checkpoint, num_labels):
        super(SpanClassificationModel, self).__init__()

        # Span extraction base model
        self.span_model = BertModel.from_pretrained(span_model_checkpoint)

        # Classification head
        self.classification_head = nn.Linear(self.span_model.config.hidden_size, num_labels)
        self.start_token_head = nn.Linear(self.span_model.config.hidden_size, 1)
        self.end_token_head = nn.Linear(self.span_model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        # Span extraction forward pass
        span_outputs = self.span_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # start_positions=start_positions,
            # end_positions=end_positions
        )

        # Get logits for start and end positions
        # start_logits = span_outputs.start_logits
        # end_logits = span_outputs.end_logits

        # get logits from start head
        start_logits = self.start_token_head(span_outputs.last_hidden_state).view(-1, span_outputs.last_hidden_state.shape[1])
        end_logits = self.end_token_head(span_outputs.last_hidden_state).view(-1, span_outputs.last_hidden_state.shape[1])

        # Apply softmax to get probabilities
        start_probs = torch.softmax(start_logits, dim=-1)
        end_probs = torch.softmax(end_logits, dim=-1)

        # Classification forward pass
        cls_logits = self.classification_head(span_outputs.last_hidden_state[:, 0, :])
        # print (cls_logits.shape)
        cls_probs = torch.softmax(cls_logits, dim=-1)
        # print (start_probs.shape, end_probs.shape, cls_probs.shape)

        return start_probs, end_probs, cls_probs

# Example usage:
span_model_checkpoint = "prajjwal1/bert-small"
classification_model_checkpoint = "prajjwal1/bert-small"
num_labels = 7  # Number of classes for text classification

model = SpanClassificationModel(span_model_checkpoint, classification_model_checkpoint, num_labels)
model.span_model.resize_token_embeddings(len(tokenizer))

# Assuming you have tokenized input (input_ids and attention_mask)
# input_ids = ...
# attention_mask = ...

# Forward pass to get start and end probabilities and classification probabilities
# start_probs, end_probs, cls_probs = model(input_ids, attention_mask)

# Find the top-k start and end positions based on probabilities (similar to previous example)
# k = 3
# top_start_probs, top_start_indices = torch.topk(start_probs, k, dim=-1)
# top_end_probs, top_end_indices = torch.topk(end_probs, k, dim=-1)

# Now, you have multiple possible spans and classification probabilities
# You can further post-process these indices and probabilities based on your specific requirements.


Embedding(30530, 512)

In [ ]:
span_model = RobertaModel.from_pretrained(span_model_checkpoint)

Some weights of RobertaModel were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['embeddings.token_type_embeddings.weight', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.3.attention.self.key.weight', 'encoder.layer.1.output.LayerNorm.weight', 'embeddings.LayerNorm.bias', 'encoder.layer.2.attention.self.value.bias', 'embeddings.word_embeddings.weight', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.1.attention.output.LayerNorm.bias', 'encoder.layer.2.attention.self.value.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.3.attention.output.dense.weight', 'encoder.layer.3.attention.output.dense.bias', 'encoder.layer.1.attention.output.dense.weight', 'encoder.layer.3.intermediate.dense.weight', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.1.attention.output.dense.bias', 'encoder.layer.2.attention.self.key.bias', 'encoder.layer.3.attention.output.Lay

In [ ]:
span_model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768, padding_idx=0)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropou

In [ ]:
span_outputs = span_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

In [ ]:
span_outputs.last_hidden_state.shape[1]

10

In [ ]:
# Example usage:
# span_model_checkpoint = "distilbert-base-uncased-distilled-squad"
# classification_model_checkpoint = "distilbert-base-uncased"
num_labels = 7  # Number of classes for text classification

# Load model and tokenizer
# model = SpanClassificationModel(span_model_checkpoint, classification_model_checkpoint, num_labels)
# tokenizer = AutoTokenizer.from_pretrained(span_model_checkpoint)

# # Add new tokens
# new_tokens = ["[NEW_TOKEN1]", "[NEW_TOKEN2]"]
# tokenizer.add_tokens(new_tokens)

# Resize the embedding layer of the base model to accommodate the new tokens
model.span_model.resize_token_embeddings(len(tokenizer))

# Assuming you have tokenized input (input_ids and attention_mask)
input_ids = tokenizer(["Your input text goes here okay?.", "Your input text goes here okay?."], return_tensors="pt")['input_ids']
attention_mask = torch.ones_like(input_ids)

# Forward pass to get start and end probabilities and classification probabilities
start_probs, end_probs, cls_probs = model(input_ids, attention_mask)

# Find the top-k start and end positions based on probabilities (similar to previous example)
k = 3
top_start_probs, top_start_indices = torch.topk(start_probs, k, dim=-1)
top_end_probs, top_end_indices = torch.topk(end_probs, k, dim=-1)

# Now, you have multiple possible spans and classification probabilities
# You can further post-process these indices and probabilities based on your specific requirements.

torch.Size([2, 7])


In [ ]:
input_ids

tensor([[ 101, 2115, 7953, 3793, 3632, 2182, 3100, 1029, 1012,  102],
        [ 101, 2115, 7953, 3793, 3632, 2182, 3100, 1029, 1012,  102]])

In [ ]:
def decode1(conversation):
    emotions = [i['emotion'] for i in conversation['conversation']]
    texts = [i['text'] for i in conversation['conversation']]
    speakers = [i['speaker'] for i in conversation['conversation']]
    speaker_dict = {}
    ind = 1
    for speaker in speakers:
        if speaker not in speaker_dict.keys():
            if ind <= 7:
                speaker_dict[speaker] = '[SP'+str(ind)+']'
                ind = ind+1
            else:
                speaker_dict[speaker] = '[SP8]'
    speakers = [speaker_dict[speaker] for speaker in speakers]

    history = [speakers[i]+conversation['conversation'][i]['text'] for i in range(0,len(texts))]
    history = "[CLS]"+ "[SEP]".join(history)
    conversation_history = [history]*len(texts)
    batch = {'emotion':emotions,"target_utterance":texts,
             "conversation_history":conversation_history}
    return batch

def decode2(conversation):
    ids = [i['utterance_ID'] for i in conversation['conversation']]
    emotions = [i['emotion'] for i in conversation['conversation']]
    texts = [i['text'] for i in conversation['conversation']]
    speakers = [i['speaker'] for i in conversation['conversation']]
    speaker_dict = {}
    ind = 1
    for speaker in speakers:
        if speaker not in speaker_dict.keys():
            if ind <= 7:
                speaker_dict[speaker] = '[SP'+str(ind)+']'
                ind = ind+1
            else:
                speaker_dict[speaker] = '[SP8]'
    speakers = [speaker_dict[speaker] for speaker in speakers]
    #speaker_no = [int(speaker[4]) for speaker in speakers]

    history = [speakers[i]+conversation['conversation'][i]['text'] for i in range(0,len(texts))]
    history = "[SEP]".join(history)
    conversation_history = [history]*len(texts)

    ec_pairs = conversation['emotion-cause_pairs']
    cause_spans = {}
    for index in ids:
        cause_spans[index-1] = []
    for ec in ec_pairs:
        utt_id = int(ec[1][:ec[1].find('_')]) - 1
        utt_text = texts[utt_id]
        text_int = utt_text.find(ec[1][ec[1].find('_')+1:])
        span_start = history.find(utt_text)+text_int
        span_end = history.find(utt_text)+text_int+len(ec[1][ec[1].find('_')+1:])
        cause_spans[int(ec[0][:ec[0].find('_')]) - 1].append((int(ec[1][:ec[1].find('_')]) - 1, span_start, span_end))
    batch = {'emotion':emotions,"target_utterance":texts,
             "conversation_history":conversation_history,"cause_spans":cause_spans}
    return batch




In [ ]:
em_dict = {'anger':0, 'disgust':1, 'fear':2, 'joy':3, 'neutral':4, 'sadness':5, 'surprise':6}

In [ ]:
target_utt_list = []
context_list = []
cause_spans_list =[]
emo_list = []
for conv in data:
  conv_decoded = decode2(conv)
  context = conv_decoded['conversation_history'][0]
  for i, utt in enumerate(conv_decoded['target_utterance']):
    target_utt_list.append(utt)
    context_list.append(context)
    cause_spans_list.append(conv_decoded['cause_spans'][i])
    emo_list.append(em_dict[conv_decoded['emotion'][i]])

In [ ]:
set(emo_list)

{0, 1, 2, 3, 4, 5, 6}

In [ ]:
decode2(data[11])['conversation_history'][0][454:558]

'We are sorry for the inconvenience , and hope you will accept this football phone ... as our free gift .'

In [ ]:
cause_spans_list[:5]

[[],
 [],
 [(0, 96, 126), (2, 174, 235)],
 [(0, 96, 126), (2, 174, 235), (3, 245, 259)],
 [(0, 96, 126), (2, 174, 235), (3, 245, 259)]]

In [ ]:
train_target_utt, val_target_utt, train_context, val_context, train_emo, val_emo, train_cause_spans, val_cause_spans \
                      = train_test_split(target_utt_list, context_list, emo_list, cause_spans_list, test_size=0.2)

In [ ]:
def char_span_to_token_span(question, context, char_start, char_end, tokenizer):
    """
    Convert character-based span to token-based span in a tokenized question and context.

    Parameters:
    - question (str): The question text.
    - context (str): The original context text.
    - char_start (int): The start index of the span in characters relative to the start of the context.
    - char_end (int): The end index of the span in characters relative to the start of the context.
    - tokenizer: The tokenizer object used to tokenize the question and context.

    Returns:
    - token_start (int): The start index of the span in tokens.
    - token_end (int): The end index of the span in tokens.
    """

    # Tokenize the question and context
    char_start = len(context[:char_start].replace(' ', ''))
    char_end = len(context[:char_end].replace(' ', ''))
    # print (char_start, char_end)

    tokens = tokenizer.tokenize(context, return_tensors="pt")
    # tokens = encoding["input_ids"].squeeze().tolist()

    # Get the length of the question
    question_len = len(tokenizer.tokenize(question, return_tensors="pt"))
    # print (question_len)

    # Initialize variables

    token_start = None
    token_end = None
    current_char = 0
    current_token = 0

    # Iterate through tokens and find the corresponding span
    while current_token < len(tokens):
        # print ("*********************************")
        # token = tokenizer.decode(tokens[current_token], skip_special_tokens=False)
        token = tokens[current_token]
        # print (token)
        # print (len(token))

        # Check if the current character is within the span
        if current_char >= char_start and current_char < char_end:
            if token_start is None:
                token_start = current_token + question_len + 2 # Adjust for the question tokens
                # print ('I am start')

            # Check if the next character goes beyond the span
            if current_char + len(token.replace('#', '')) >= char_end:
                token_end = current_token + question_len + 2 # Adjust for the question tokens
                # print ('I am end')
                break

        # Move to the next token and update character position
        current_char += len(token.replace('#', ''))
        # print (current_char)
        current_token += 1
        # print (current_token)

    return token_start, token_end

# Example usage:
# question = "That is one way !"
# context = '[SP1]Oh my God ![SEP][SP2]I know , I know , I am such an idiot .[SEP][SP2]I guess I should have caught on when she started going to the dentist four and five times a week . I mean , how clean can teeth get ?[SEP][SP1]My brother going through that right now , he is such a mess . How did you get through it ?[SEP][SP2]Well , you might try accidentally breaking something valuable of hers , say her ...[SEP][SP1]leg ?[SEP][SP2]That is one way ![SEP][SP2]Me , I ... I went for the watch .[SEP][SP1]You actually broke her watch ?'
question = target_utt_list[696]
context = context_list[696]
char_start = 411
char_end = 477

# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
token_start, token_end = char_span_to_token_span(question, context, char_start, char_end, tokenizer)

print("Token Start:", token_start)
print("Token End:", token_end)

Token Start: 124
Token End: 136


In [ ]:
BATCH_SIZE = 16

batch_encoded_conversation = []
batch_cause_spans = []
batch_start_token_indx = []
batch_end_token_indx = []

batch_emo_label = []

for i in range(0, len(target_utt_list), BATCH_SIZE):
    start_end_token_indx = []
    batch_encoded_conversation.append(tokenizer(target_utt_list[i: i + BATCH_SIZE], context_list[i :i + BATCH_SIZE],
                                                return_tensors="pt", padding=True, max_length=512, truncation=True,
                                                return_token_type_ids=False))
    batch_emo_label.append(torch.tensor(emo_list[i: i + BATCH_SIZE], dtype=torch.long))
    batch_cause_spans.append(cause_spans_list[i:i + BATCH_SIZE])
    start_token_indx = []
    end_token_indx = []
    start_token_indx = np.zeros([len(target_utt_list[i: i + BATCH_SIZE]), int(batch_encoded_conversation[-1]['input_ids'].shape[1])])
    end_token_indx = np.zeros([len(target_utt_list[i: i + BATCH_SIZE]), int(batch_encoded_conversation[-1]['input_ids'].shape[1])])
    for j in range(len(target_utt_list[i: i + BATCH_SIZE])):
          se_indx = [char_span_to_token_span(target_utt_list[i + j], context_list[i + j], cause_span[1], cause_span[2], tokenizer) for cause_span in cause_spans_list[i + j]]
          if len(se_indx) > 0:
              # print (i + j, se_indx)
              s_indx = [int(x[0]) for x in se_indx if x[0] is not None and x[0] < 512]
              e_indx = [int(x[1]) for x in se_indx if x[1] is not None and x[1] < 512]
              # print(s_indx)
              # print(e_indx)
              start_token_indx[j, s_indx] = 1.
              end_token_indx[j, e_indx] = 1.
              # print(start_token_indx[j,:])

          # start_token_indx.append([x[0] for x in se_indx])
          # end_token_indx.append([x[1] for x in se_indx])
    batch_start_token_indx.append(torch.tensor(start_token_indx, dtype=torch.long))
    batch_end_token_indx.append(torch.tensor(end_token_indx, dtype=torch.long))


KeyboardInterrupt: ignored

In [ ]:
def create_batches(target_utt_list, context_list, emo_list, cause_spans_list):
    batch_encoded_conversation = []
    batch_cause_spans = []
    batch_start_token_indx = []
    batch_end_token_indx = []

    batch_emo_label = []

    for i in range(0, len(target_utt_list), BATCH_SIZE):
        start_end_token_indx = []
        batch_encoded_conversation.append(tokenizer(target_utt_list[i: i + BATCH_SIZE], context_list[i :i + BATCH_SIZE],
                                                    return_tensors="pt", padding=True, max_length=512, truncation=True,
                                                    return_token_type_ids=False))
        batch_emo_label.append(torch.tensor(emo_list[i: i + BATCH_SIZE], dtype=torch.long))
        batch_cause_spans.append(cause_spans_list[i:i + BATCH_SIZE])
        start_token_indx = []
        end_token_indx = []
        start_token_indx = np.zeros([len(target_utt_list[i: i + BATCH_SIZE]), int(batch_encoded_conversation[-1]['input_ids'].shape[1])])
        end_token_indx = np.zeros([len(target_utt_list[i: i + BATCH_SIZE]), int(batch_encoded_conversation[-1]['input_ids'].shape[1])])
        for j in range(len(target_utt_list[i: i + BATCH_SIZE])):
              se_indx = [char_span_to_token_span(target_utt_list[i + j], context_list[i + j], cause_span[1], cause_span[2], tokenizer) for cause_span in cause_spans_list[i + j]]
              if len(se_indx) > 0:
                  # print (i + j, se_indx)
                  s_indx = [int(x[0]) for x in se_indx if x[0] is not None and x[0] < 512]
                  e_indx = [int(x[1]) for x in se_indx if x[1] is not None and x[1] < 512]
                  # print(s_indx)
                  # print(e_indx)
                  start_token_indx[j, s_indx] = 1.
                  end_token_indx[j, e_indx] = 1.
                  # print(start_token_indx[j,:])

              # start_token_indx.append([x[0] for x in se_indx])
              # end_token_indx.append([x[1] for x in se_indx])
        batch_start_token_indx.append(torch.tensor(start_token_indx, dtype=torch.float))
        batch_end_token_indx.append(torch.tensor(end_token_indx, dtype=torch.float))
    return batch_encoded_conversation, batch_emo_label, batch_start_token_indx, batch_end_token_indx

In [ ]:
BATCH_SIZE = 16


In [ ]:
train_batch_conv, train_batch_emo, train_batch_start, train_batch_end = create_batches(train_target_utt, train_context, train_emo, train_cause_spans)

In [ ]:
len(train_batch_conv)

681

In [ ]:
val_batch_conv, val_batch_emo, val_batch_start, val_batch_end = create_batches(val_target_utt, val_context, val_emo, val_cause_spans)

In [ ]:
tokenizer.decode(train_batch_conv[3]['input_ids'][10][155:159])

'the cast of cats'

In [ ]:
torch.where(batch_start_token_indx[3][10]==1), torch.where(batch_end_token_indx[3][10]==1)

((tensor([122, 141, 155]),), (tensor([138, 148, 158]),))

In [ ]:
context_list[58][533:548]

'that is great .'

In [ ]:
cause_spans_list[58]

[(10, 408, 464), (11, 474, 508), (12, 533, 548)]

In [ ]:
batch_encoded_conversation[0]

{'input_ids': tensor([[  101, 10303,  1010,  ...,     0,     0,     0],
        [  101,  2821,  1010,  ...,     0,     0,     0],
        [  101,  2059,  1045,  ...,     0,     0,     0],
        ...,
        [  101,  1045,  3984,  ...,  3422,  1029,   102],
        [  101,  2026,  2567,  ...,     0,     0,     0],
        [  101,  2092,  1010,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
242 - 64*3

50

In [ ]:
batch_start_token_indx[3][50],batch_end_token_indx[3][50]

IndexError: ignored

In [ ]:
target_utt_list[242]

'Yes , how about . C mere . Gimme ! Give me my puck .'

In [ ]:
tokenizer.decode(batch_encoded_conversation[1]['input_ids'][23][153:176])

' for the inconvenience, and hope you will accept this football phone... as our free gift. Do you believe this?'

In [ ]:
tokenizer.decode(batch_encoded_conversation[0]['input_ids'][13])

'[CLS] i guess i should have caught on when she started going to the dentist four and five times a week. i mean, how clean can teeth get? [SEP] [SP1] oh my god! [SEP] [SP2] i know, i know, i am such an idiot. [SEP] [SP2] i guess i should have caught on when she started going to the dentist four and five times a week. i mean, how clean can teeth get? [SEP] [SP1] my brother going through that right now, he is such a mess. how did you get through it? [SEP] [SP2] well, you might try accidentally breaking something valuable of hers, say her... [SEP] [SP1] leg? [SEP] [SP2] that is one way! [SEP] [SP2] me, i... i went for the watch. [SEP] [SP1] you actually broke her watch? [SEP]'

In [ ]:
batch_emo_label[0]

[4, 4, 6, 6, 0, 4, 4, 4, 5, 4, 6, 6, 4, 5, 4, 4]

In [ ]:
batch_encoded_conversation[0]['input_ids'].shape[1]

159

In [ ]:
cause_spans_list[242]

[(15, 843, 863), (16, 873, 877)]

In [ ]:
context_list[242][843:863]

'gimme back my puck .'

In [ ]:
tokenizer

In [ ]:
tokenizer.decode(tokenizer(context_list[242])['input_ids'])

'[CLS] [SP1] man. can you believe he is only had sex with one woman? [SEP] [SP2] i think it is great. you know, it is sweet, it is romantic... [SEP] [SP1] really? [SEP] [SP2] no, you kidding? the guy a freak. i judge him. [SEP] [SP3] hey, buddy. [SEP] [SP4] oh, that is attractive. [SEP] [SP2] oh, i thought you were great in silence of the lambs. [SEP] [SP2] oh come on, admit it! all things considered, you had fun tonight. [SEP] [SP5] fun? where was the fun? tell me specifically, which part was the fun part? where is my puck? [SEP] [SP1] oh, ah... the kid has it. [SEP] [SP5] the kid...? [SEP] [SP5] excuse me, uh, that is, that is my puck. [SEP] [SP6] i found it. finders keepers, losers weepers. [SEP] [SP2] you gotta do it, man. [SEP] [SP5] oh yeah? well, i am rubber, you are glue, whatever you... can not do it. [SEP] [SP5] listen, uh... gimme back my puck. [SEP] [SP6] no! [SEP] [SP5] yes, how about. c mere. gimme! give me my puck. [SEP] [SP4] hey! hey! no rough holding in my er! [SEP] [

In [ ]:
tokenizer.decode(tokenizer(['tezt 1', 'hcbi'], ['text2', 'test2'])['input_ids'][1])

'<s>hcbi</s></s>test2</s>'

In [ ]:
batch_cause_spans[1][23]

[(7, 387, 444), (8, 454, 558), (8, 583, 637), (8, 454, 558), (8, 583, 637)]

In [ ]:
decode2(data[2])

{'emotion': ['surprise',
  'neutral',
  'sadness',
  'neutral',
  'neutral',
  'surprise',
  'joy',
  'neutral',
  'surprise'],
 'target_utterance': ['Oh my God !',
  'I know , I know , I am such an idiot .',
  'I guess I should have caught on when she started going to the dentist four and five times a week . I mean , how clean can teeth get ?',
  'My brother going through that right now , he is such a mess . How did you get through it ?',
  'Well , you might try accidentally breaking something valuable of hers , say her ...',
  'leg ?',
  'That is one way !',
  'Me , I ... I went for the watch .',
  'You actually broke her watch ?'],
 'conversation_history': ['[SP1]Oh my God ![SEP][SP2]I know , I know , I am such an idiot .[SEP][SP2]I guess I should have caught on when she started going to the dentist four and five times a week . I mean , how clean can teeth get ?[SEP][SP1]My brother going through that right now , he is such a mess . How did you get through it ?[SEP][SP2]Well , you mi

In [ ]:
a = '[SP1]Oh my God ![SEP][SP2]I know , I know , I am such an idiot .[SEP][SP2]I guess I should have caught on when she started going to the dentist four and five times a week . I mean , how clean can teeth get ?[SEP][SP1]My brother going through that right now , he is such a mess . How did you get through it ?[SEP][SP2]Well , you might try accidentally breaking something valuable of hers , say her ...[SEP][SP1]leg ?[SEP][SP2]That is one way ![SEP][SP2]Me , I ... I went for the watch .[SEP][SP1]You actually broke her watch ?'
a[425:442]

'That is one way !'

In [ ]:
tokenizer.decode(batch_encoded_conversation[0]['input_ids'][0])

'[CLS] alright, so i am back in high school, i am standing in the middle of the cafeteria, and i realize i am totally naked. [SEP] [SP1] alright, so i am back in high school, i am standing in the middle of the cafeteria, and i realize i am totally naked. [SEP] [SP2] oh, yeah. had that dream. [SEP] [SP1] then i look down, and i realize there is a phone... there. [SEP] [SP3] instead of...? [SEP] [SP1] that is right. [SEP] [SP3] never had that dream. [SEP] [SP4] no. [SEP] [SP1] all of a sudden, the phone starts to ring. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [ ]:
batch_encoded_conversation[0]['offset_mapping'][0]

tensor([[  0,   0],
        [  0,   7],
        [  8,   9],
        [ 10,  12],
        [ 13,  14],
        [ 15,  17],
        [ 18,  22],
        [ 23,  25],
        [ 26,  30],
        [ 31,  37],
        [ 38,  39],
        [ 40,  41],
        [ 42,  44],
        [ 45,  53],
        [ 54,  56],
        [ 57,  60],
        [ 61,  67],
        [ 68,  70],
        [ 71,  74],
        [ 75,  84],
        [ 85,  86],
        [ 87,  90],
        [ 91,  92],
        [ 93, 100],
        [101, 102],
        [103, 105],
        [106, 113],
        [114, 119],
        [120, 121],
        [  0,   0],
        [  0,   5],
        [  5,  12],
        [ 13,  14],
        [ 15,  17],
        [ 18,  19],
        [ 20,  22],
        [ 23,  27],
        [ 28,  30],
        [ 31,  35],
        [ 36,  42],
        [ 43,  44],
        [ 45,  46],
        [ 47,  49],
        [ 50,  58],
        [ 59,  61],
        [ 62,  65],
        [ 66,  72],
        [ 73,  75],
        [ 76,  79],
        [ 80,  89],


In [ ]:
def char_span_to_token_span(question, context, char_start, char_end, tokenizer):
    """
    Convert character-based span to token-based span in a tokenized question and context.

    Parameters:
    - question (str): The question text.
    - context (str): The original context text.
    - char_start (int): The start index of the span in characters relative to the start of the context.
    - char_end (int): The end index of the span in characters relative to the start of the context.
    - tokenizer: The tokenizer object used to tokenize the question and context.

    Returns:
    - token_start (int): The start index of the span in tokens.
    - token_end (int): The end index of the span in tokens.
    """

    # Tokenize the question and context
    char_start = len(context[:char_start].replace(' ', ''))
    char_end = len(context[:char_end].replace(' ', ''))
    # print (char_start, char_end)

    tokens = tokenizer.tokenize(context, return_tensors="pt")
    # tokens = encoding["input_ids"].squeeze().tolist()

    # Get the length of the question
    question_len = len(tokenizer.tokenize(question, return_tensors="pt"))
    # print (question_len)

    # Initialize variables

    token_start = None
    token_end = None
    current_char = 0
    current_token = 0

    # Iterate through tokens and find the corresponding span
    while current_token < len(tokens):
        # print ("*********************************")
        # token = tokenizer.decode(tokens[current_token], skip_special_tokens=False)
        token = tokens[current_token]
        # print (token)
        # print (len(token))

        # Check if the current character is within the span
        if current_char >= char_start and current_char < char_end:
            if token_start is None:
                token_start = current_token + question_len + 2 # Adjust for the question tokens
                # print ('I am start')

            # Check if the next character goes beyond the span
            if current_char + len(token.replace('#', '')) >= char_end:
                token_end = current_token + question_len + 2 # Adjust for the question tokens
                # print ('I am end')
                break

        # Move to the next token and update character position
        current_char += len(token.replace('#', ''))
        # print (current_char)
        current_token += 1
        # print (current_token)

    return token_start, token_end

# Example usage:
# question = "That is one way !"
# context = '[SP1]Oh my God ![SEP][SP2]I know , I know , I am such an idiot .[SEP][SP2]I guess I should have caught on when she started going to the dentist four and five times a week . I mean , how clean can teeth get ?[SEP][SP1]My brother going through that right now , he is such a mess . How did you get through it ?[SEP][SP2]Well , you might try accidentally breaking something valuable of hers , say her ...[SEP][SP1]leg ?[SEP][SP2]That is one way ![SEP][SP2]Me , I ... I went for the watch .[SEP][SP1]You actually broke her watch ?'
question = target_utt_list[696]
context = context_list[696]
char_start = 411
char_end = 477

# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
token_start, token_end = char_span_to_token_span(question, context, char_start, char_end, tokenizer)

print("Token Start:", token_start)
print("Token End:", token_end)

Token Start: 124
Token End: 136


In [ ]:
tokenizer.decode(tokenizer(question, context, return_tensors='pt')['input_ids'][0, 124:137])

'forget about the birds, but big red insects suggest fine dining!'

In [ ]:
print (question)
print (context)

['What is an example?', 'This is an example sentence.']
[SP1]I never sucked , I actually did not want you to know how good I was ![SEP][SP2]Why ?[SEP][SP1]I do not know .[SEP][SP2]This is so great ! Now we can enter into doubles tournaments ![SEP][SP1]That is why !


In [ ]:
print (target_utt_list[696])
print (context_list[696])

Oh , so , forget about the birds , but big red insects suggest fine dining !
[SP1]Will you pick one , just pick one ! Here , how about that one ?[SEP][SP2]That is patio furniture ![SEP][SP1]So what , like people are gonna come in and think , " Uh ... oh , I am outside again ? "[SEP][SP2]What about the birds ?[SEP][SP1]I do not know , birds just do not say , " Hello , sit here , eat something . "[SEP][SP2]You pick one .[SEP][SP1]All right , how about the ladybugs ?[SEP][SP2]Oh , so , forget about the birds , but big red insects suggest fine dining ![SEP][SP1]Fine , you want to get the birds , get the birds ![SEP][SP2]Not like that , I will not ![SEP][SP2]Kip would have liked the birds !


In [ ]:
cause_spans_list[696]

[(3, 211, 233), (4, 243, 321), (6, 367, 391), (7, 411, 477)]

In [ ]:
context_list[696][411:477]

'forget about the birds , but big red insects suggest fine dining !'

In [ ]:
context_list[696].replace(' ', '')[411:477]

'etthebirds,getthebirds![SEP][SP2]Notlikethat,Iwillnot![SEP][SP2]Ki'

In [ ]:
tokenizer.decode(tokenizer(question)['input_ids'])

'[CLS] ross? that opens my cervix. [SEP]'

In [ ]:
tokenizer.decode(tokenizer(context, return_tensors="pt")['input_ids'][0])

'[CLS] [SP1] alright, so i am back in high school, i am standing in the middle of the cafeteria, and i realize i am totally naked. [SEP] [SP2] oh, yeah. had that dream. [SEP] [SP1] then i look down, and i realize there is a phone... there. [SEP] [SP3] instead of...? [SEP] [SP1] that is right. [SEP] [SP3] never had that dream. [SEP] [SP4] no. [SEP] [SP1] all of a sudden, the phone starts to ring. [SEP]'

In [ ]:
len(context[:96].replace(' ', ''))

75

In [ ]:
context[:126].replace(' ', '')

'[SP1]Letitgo,Ross.[SEP][SP2]Yeah,well,youdidnotknowChiChi.[SEP][SP3]Doyouallpromise?[SEP][SP4]Yeah!Wepro'

In [ ]:
token_start, token_end = char_span_to_token_span(question, context, char_start, char_end, tokenizer)


In [ ]:
for param in model.parameters():
     param.requires_grad = True

# New Attempt

In [ ]:
# Imports for most of the notebook
import torch
from transformers import BertModel
from transformers import AutoTokenizer
from typing import Dict, List
import random
from tqdm.autonotebook import tqdm

In [ ]:
print(torch.cuda.is_available())
# device = torch.device("cpu")
# TODO: Uncomment the below line if you see True in the print statement
device = torch.device("cuda:0")

True


In [ ]:
class NLIClassifier(torch.nn.Module):
    def __init__(self, output_size: int, hidden_size: int, model_name='Nakul24/Spanbert-emotion-extraction'):
        super().__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size

        # Initialize BERT, which we use instead of a single embedding layer.
        self.bert = BertModel.from_pretrained(model_name)

        # TODO [OPTIONAL]: Updating all BERT parameters can be slow and memory intensive.
        # Freeze them if training is too slow. Notice that the learning
        # rate should probably be smaller in this case.
        # Uncommenting out the below 2 lines means only our classification layer will be updated.

        # for param in self.bert.parameters():
        #     param.requires_grad = False

        self.bert_hidden_dimension = self.bert.config.hidden_size

        # TODO: Add an extra hidden layer in the classifier, projecting
        #      from the BERT hidden dimension to hidden size. Hint: torch.nn.Linear()

        self.hidden_layer = torch.nn.Linear(self.bert_hidden_dimension, self.hidden_size)

        # TODO: Add a relu nonlinearity to be used in the forward method
        #      https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html

        self.relu = torch.nn.ReLU()

        self.classifier = torch.nn.Linear(self.hidden_size, self.output_size)
        self.log_softmax = torch.nn.LogSoftmax(dim=2)

        self.span_start_classifier = torch.nn.Linear(self.bert_hidden_dimension, 1)
        self.softmax_starts = torch.nn.Softmax(dim=1)

        self.span_end_classifier = torch.nn.Linear(self.bert_hidden_dimension, 1)
        self.softmax_ends = torch.nn.Softmax(dim=1)

    def encode_text(
        self,
        symbols: Dict
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols BERT.
            Then, get CLS represenation.

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: CLS token embedding
        """
        # First we get the contextualized embedding for each input symbol
        # We no longer need an LSTM, since BERT encodes context and
        # gives us a single vector describing the sequence in the form of the [CLS] token.
        encoded_sequence = self.bert(**symbols)
        # TODO: Get the [CLS] token
        #      The BertModel output. See here: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        #      and check the returns for the forward method.
        # We want to return a tensor of the form batch_size x 1 x bert_hidden_dimension
        # print(encoded_sequence.last_hidden_state[:, :1, :].shape)
        # Return only the first token's embedding from the last_hidden_state. Hint: using list slices
        return encoded_sequence.last_hidden_state[:, :1, :]

    def forward(
        self,
        symbols: Dict,
    ) -> torch.Tensor:
        """_summary_

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: _description_
        """
        encoded_sequence = self.bert(**symbols)

        encoded_sents = encoded_sequence.last_hidden_state[:, :1, :]
        # encoded_sents = self.encode_text(symbols)
        output = self.hidden_layer(encoded_sents)
        output = self.relu(output)
        output = self.classifier(output)

        span_start_logits = self.span_start_classifier(encoded_sequence.last_hidden_state)
        start_probs = self.softmax_starts(span_start_logits)
        # print (start_probs.shape)
        span_end_logits = self.span_start_classifier(encoded_sequence.last_hidden_state)
        end_probs = self.softmax_ends(span_end_logits)

        return self.log_softmax(output), start_probs, end_probs

In [ ]:
# For making predictions at test time
def predict(model: torch.nn.Module, sents: torch.Tensor) -> List:
    logits, start_probs, end_probs = model(sents)
    return list(torch.argmax(logits, axis=2).squeeze().cpu().numpy()),\
            start_probs.squeeze(2).cpu().detach().numpy(), end_probs.squeeze(2).cpu().detach().numpy()

In [ ]:
import numpy as np
from numpy import sum as t_sum
from numpy import logical_and


def precision(predicted_labels, true_labels, which_label=1):
    """
    Precision is True Positives / All Positives Predictions
    """
    pred_which = np.array([pred == which_label for pred in predicted_labels])
    true_which = np.array([lab == which_label for lab in true_labels])
    denominator = t_sum(pred_which)
    if denominator:
        return t_sum(logical_and(pred_which, true_which))/denominator
    else:
        return 0.


def recall(predicted_labels, true_labels, which_label=1):
    """
    Recall is True Positives / All Positive Labels
    """
    pred_which = np.array([pred == which_label for pred in predicted_labels])
    true_which = np.array([lab == which_label for lab in true_labels])
    denominator = t_sum(true_which)
    if denominator:
        return t_sum(logical_and(pred_which, true_which))/denominator
    else:
        return 0.


def f1_score(
    predicted_labels: List[int],
    true_labels: List[int],
    which_label: int
):
    """
    F1 score is the harmonic mean of precision and recall
    """
    P = precision(predicted_labels, true_labels, which_label=which_label)
    R = recall(predicted_labels, true_labels, which_label=which_label)

    if P and R:
        return 2*P*R/(P+R)
    else:
        return 0.


def macro_f1(
    predicted_labels: List[int],
    true_labels: List[int],
    possible_labels: List[int],
    label_map=None
):
    converted_prediction = [label_map[int(x)] for x in predicted_labels] if label_map else predicted_labels
    scores = [f1_score(converted_prediction, true_labels, l) for l in possible_labels]
    # Macro, so we take the uniform avg.
    return sum(scores) / len(scores)

In [ ]:
def training_loop(
    num_epochs,
    train_features,
    train_labels,
    train_span_starts,
    train_span_ends,
    dev_sents,
    dev_labels,
    dev_span_starts,
    dev_span_ends,
    optimizer,
    model,
):
    print("Training...")
    #weight = torch.tensor([2,3,3,2,0.5,2,2])
    loss_func = torch.nn.NLLLoss()
    loss_cross_ent = torch.nn.CrossEntropyLoss()
    batches = list(zip(train_features, train_labels, train_span_starts, train_span_ends))
    random.shuffle(batches)

    train_labels_all = list(torch.concat(list(zip(*batches))[1]).numpy())
    train_span_starts = list(zip(*batches))[2]
    train_span_ends = list(zip(*batches))[3]

    train_starts_all = torch.concat([x.view(-1) for x in train_span_starts]).numpy()
    train_ends_all = torch.concat([x.view(-1) for x in train_span_ends]).numpy()

    dev_span_starts_all = torch.concat([x.view(-1) for x in dev_span_starts]).numpy()
    dev_span_ends_all = torch.concat([x.view(-1) for x in dev_span_ends]).numpy()

    possible_labels = set(train_labels_all)

    for i in range(num_epochs):
        losses = []
        train_preds = []
        train_start_probs = []
        train_end_probs = []
        for features, labels, span_starts, span_ends in tqdm(batches):
            # Empty the dynamic computation graph
            optimizer.zero_grad()
            preds, start_probs, end_probs = model(features.to(device))
            preds, start_probs, end_probs = \
                  preds.squeeze(1), start_probs.squeeze(2), end_probs.squeeze(2)
            loss_emo = loss_func(preds, labels.to(device))
            loss_start = loss_cross_ent(start_probs, span_starts.to(device))
            loss_end = loss_cross_ent(end_probs, span_ends.to(device))

            loss = loss_emo + loss_start + loss_end

            # Backpropogate the loss through our model
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            train_preds.append(preds)
            train_start_probs.append(start_probs)
            train_end_probs.append(end_probs)

        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")
        print ("Evaluating train...")
        train_preds = torch.concat(train_preds)
        # print (train_preds.shape)
        train_preds = list(torch.argmax(train_preds, axis=1).squeeze().cpu().numpy())
        # train_labels_all = list(torch.concat(train_labels_all).cpu().numpy())
        train_f1 = macro_f1(train_preds, train_labels_all, possible_labels)
        print(f"Train F1 {train_f1}")

        train_start_probs = torch.concat([x.view(-1) for x in train_start_probs]).cpu().detach().numpy()
        train_start_auc = roc_auc_score(train_starts_all, train_start_probs)
        print(f"Train START AUC: {train_start_auc}")

        train_end_probs = torch.concat([x.view(-1) for x in train_end_probs]).cpu().detach().numpy()
        train_end_auc = roc_auc_score(train_ends_all, train_end_probs)
        print(f"Train END AUC: {train_end_auc}")


        # Estimate the f1 score for the development set
        print("Evaluating dev...")
        all_preds = []
        all_labels = []
        all_starts = []
        all_ends = []
        for sents, labels in tqdm(zip(dev_sents, dev_labels), total=len(dev_sents)):
            # pred = predict(model, sents).cpu()
            pred, start_probs, end_probs = predict(model, sents.to(device))
            all_preds.extend(pred)
            all_labels.extend(list(labels.cpu().numpy()))
            all_starts.append(start_probs)
            all_ends.append(end_probs)

        dev_f1 = macro_f1(all_preds, all_labels, set(all_labels))
        print(f"Dev F1 {dev_f1}")

        all_starts = np.concatenate([x.reshape(-1) for x in all_starts])
        dev_start_auc = roc_auc_score(dev_span_starts_all, all_starts)
        print(f"Dev START AUC: {dev_start_auc}")

        all_ends = np.concatenate([x.reshape(-1) for x in all_ends])
        dev_end_auc = roc_auc_score(dev_span_ends_all, all_ends)
        print(f"Dev END AUC: {dev_end_auc}")

    # Return the trained model
    return model

In [ ]:
# You can increase epochs if need be
epochs = 5

# TODO: Find a good learning rate and hidden size
LR = 0.0005
hidden_size = 32

possible_labels = set(emo_list)
model = NLIClassifier(output_size=len(possible_labels), hidden_size=hidden_size)
model.bert.resize_token_embeddings(len(tokenizer))
model.to(device)
optimizer = torch.optim.AdamW([{'params': model.hidden_layer.parameters()},
                               {'params': model.classifier.parameters()},
                               {'params': model.bert.parameters(), 'lr': 0.000005}],
                              lr=LR)
# optimizer = torch.optim.AdamW(model.parameters(), lr=LR)


# batch_tokenizer = BatchTokenizer()

# validation_input_batches = [b for b in chunk_multi(validation_premises, validation_hypotheses, batch_size)]

# Tokenize + encode
# validation_input_batches = [batch_tokenizer(*batch) for batch in validation_input_batches]
# validation_batch_labels = [b for b in chunk(validation_labels, batch_size)]
# validation_batch_labels = [encode_labels(batch) for batch in validation_batch_labels]

trained_model = training_loop(
    epochs,
    train_batch_conv,
    train_batch_emo,
    train_batch_start,
    train_batch_end,
    val_batch_conv,
    val_batch_emo,
    val_batch_start,
    val_batch_end,
    optimizer,
    model,
)

torch.save(model.state_dict(), './Emo_classifier_and_span.pt')

Some weights of BertModel were not initialized from the model checkpoint at Nakul24/Spanbert-emotion-extraction and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training...


  0%|          | 0/681 [00:00<?, ?it/s]

epoch 0, loss: 9.552514840152645
Evaluating train...
Train F1 0.1539748984994514
Train START AUC: 0.7347028967050251
Train END AUC: 0.7952446086116616
Evaluating dev...


  0%|          | 0/171 [00:00<?, ?it/s]

...

Dev F1 0.4219100625609773
Dev START AUC: 0.8114708672369282
Dev END AUC: 0.9284386382185372


  0%|          | 0/681 [00:00<?, ?it/s]

epoch 4, loss: 8.576739092755423
Evaluating train...
Train F1 0.710969507494151
Train START AUC: 0.8181043811340258
Train END AUC: 0.9272877696770256
Evaluating dev...


  0%|          | 0/171 [00:00<?, ?it/s]

Dev F1 0.43464475356773846
Dev START AUC: 0.8181043811340258
Dev END AUC: 0.9370259945256416


In [ ]:
trained_model = training_loop(
    3,
    train_batch_conv,
    train_batch_emo,
    train_batch_start,
    train_batch_end,
    val_batch_conv,
    val_batch_emo,
    val_batch_start,
    val_batch_end,
    optimizer,
    model,
)

Training...


  0%|          | 0/681 [00:00<?, ?it/s]

epoch 0, loss: 8.476303671950285
Evaluating train...
Train F1 0.7607272299424707
Train START AUC: 0.8219791233620114
Train END AUC: 0.9348700029135941
Evaluating dev...


  0%|          | 0/171 [00:00<?, ?it/s]

Dev F1 0.4575349426651031
Dev START AUC: 0.8219791233620114
Dev END AUC: 0.9470746864442039


...

Dev F1 0.44869606091966985
Dev START AUC: 0.8243503719610125
Dev END AUC: 0.9514227714360466


  0%|          | 0/681 [00:00<?, ?it/s]

epoch 2, loss: 8.188285914811795
Evaluating train...
Train F1 0.9218568916272586
Train START AUC: 0.8234916599484974
Train END AUC: 0.9457963701852872
Evaluating dev...


  0%|          | 0/171 [00:00<?, ?it/s]

Dev F1 0.45621510435421675
Dev START AUC: 0.8234916599484974
Dev END AUC: 0.9540992441074787


In [ ]:
torch.save(model.state_dict(), './Emo_classifier_and_span.pt')

In [ ]:
all_preds = []
all_labels = []
all_starts = []
all_ends = []
for sents, labels in tqdm(zip(val_batch_conv, val_batch_emo), total=len(val_batch_conv)):
    # pred = predict(model, sents).cpu()
    pred, start_probs, end_probs = predict(model, sents.to(device))
    all_preds.extend(pred)
    all_labels.extend(list(labels.cpu().numpy()))
    all_starts.append(start_probs)
    all_ends.append(end_probs)

dev_f1 = macro_f1(all_preds, all_labels, set(all_labels))
print(f"Dev F1 {dev_f1}")

# all_starts = np.concatenate([x.reshape(-1) for x in all_starts])
# dev_start_auc = roc_auc_score(dev_span_starts_all, all_starts)
# print(f"Dev START AUC: {dev_start_auc}")

# all_ends = np.concatenate([x.reshape(-1) for x in all_ends])
# dev_end_auc = roc_auc_score(dev_span_ends_all, all_ends)
# print(f"Dev END AUC: {dev_end_auc}")

  0%|          | 0/171 [00:00<?, ?it/s]

In [ ]:
for batch_conv, batch_emo in zip(val_batch_conv, val_batch_emo):
    preds, preds_start, preds_end = model_eval(batch_conv)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-38-c2dd1ca139b5>", line 2, in <cell line: 1>
    preds, preds_start, preds_end = model_eval(batch_conv)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "<ipython-input-32-848a42e72166>", line 76, in forward
    encoded_sequence = self.bert(**symbols)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_

In [ ]:
torch.save(model.state_dict(), './Emo_classifier_and_span.pt')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-38-c02642bb5f4c>", line 1, in <cell line: 1>
    torch.save(model.state_dict(), './Emo_classifier_and_span.pt')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 618, in save
    with _open_zipfile_writer(f) as opened_zipfile:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 492, in _open_zipfile_writer
    return container(name_or_buffer)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 463, in __init__
    super().__init__(torch._C.PyTorchFileWriter(self.name))
RuntimeError: Parent directory . does not exist.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactivesh

In [ ]:
model = NLIClassifier(output_size=len(possible_labels), hidden_size=32)
model.bert.resize_token_embeddings(len(tokenizer))
model.to(device)
out, outs, oute = model(batch_encoded_conversation[0].to(device))

In [ ]:
out, outs, oute = predict(model, val_batch_conv[3].to(device))

In [ ]:
np.concatenate([outs.reshape(-1), outs.reshape(-1)]).shape

(7552,)

In [ ]:
def decode_output(batch_conv, batch_start, batch_end):
  output = {}
  start_indices = np.where(batch_start != 0)
  end_indices = np.where(batch_end != 0)
  for i in range(len(batch_conv['input_ids'])):
    conv = batch_conv['input_ids'][i]
    output[i] = []
    if i in start_indices[0]:
      s_idx = start_indices[1][np.where(start_indices[0] == i)]
      e_idx = end_indices[1][np.where(end_indices[0] == i)]

      utt_ids = [len([x for x in conv[:k] if x == sep_token]) for k in s_idx]
      #print(utt_ids)
      texts = [tokenizer.decode(conv[s_idx[j]:e_idx[j]+1]) for j in range(len(s_idx))]

      output[i] = [str(utt_ids[k])+'_'+texts[k] for k in range(len(texts))]
  return output



In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Nakul24/Spanbert-emotion-extraction")
model = AutoModelForQuestionAnswering.from_pretrained("Nakul24/Spanbert-emotion-extraction")

tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

In [ ]:
import torch
inp = tokenizer(['what is my weight?', 'hello'], [' My weight is 60 kg.', 'hellp'], return_tensors='pt', padding=True)

In [ ]:
out = model(**inp)

In [ ]:
torch.argmax(out.start_logits, axis=1), torch.argmax(out.end_logits, axis=1)

(tensor([4, 1]), tensor([0, 2]))

In [ ]:
inp

{'input_ids': tensor([[  101,  1184,  1110,  1139,  2841,   136,   102,  1139,  2841,  1110,
          2539,  4023,   119,   102],
        [  101, 19082,   102,  2630,  1643,   102,     0,     0,     0,     0,
             0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [ ]:
def decode_span(conversation):
  ids = [i['utterance_ID'] for i in conversation['conversation']]
  emotions = [i['emotion'] for i in conversation['conversation']]

  non_neutral_emotions = [i+1 for i in range(len(emotions)) if emotions[i] != 'neutral']

  texts = [i['text'] for i in conversation['conversation']]
  speakers = [i['speaker'] for i in conversation['conversation']]
  speaker_dict = {}
  ind = 1
  for speaker in speakers:
      if speaker not in speaker_dict.keys():
          if ind <= 7:
              speaker_dict[speaker] = '[SP'+str(ind)+']'
              ind = ind+1
          else:
              speaker_dict[speaker] = '[SP8]'
  speakers = [speaker_dict[speaker] for speaker in speakers]

  history = [speakers[i]+conversation['conversation'][i]['text'] for i in range(0,len(texts))]
  history = "[SEP]".join(history)

  ec_pairs = conversation['emotion-cause_pairs']

  ec_dict = {}
  for ind in non_neutral_emotions:
    ec_dict[ind] = []

  for ec in ec_pairs:
    ec_dict[int(ec[0].split('_')[0])].append(ec[1])

  emotion = []
  target_utterence = []
  evidence_utterence = []
  conversation_history = []
  cause_spans = []

  for i in range(0, len(texts)):
    if emotions[i] != 'neutral':
      emotion = emotion + ([emotions[i]]*len(texts))
      target_utterence = target_utterence + ([texts[i]]*len(texts))
      #evidence_utterence = evidence_utterence + texts

      conversation_history_single = [text + ' [SEP] '+ history for text in texts]

      conversation_history = conversation_history + (conversation_history_single)
      relevant_contexts = ec_dict[i+1]
      relevant_contexts_dict = {}
      for context in relevant_contexts:
        relevant_contexts_dict[int(context.split('_')[0])-1] = context.split('_')[1]
      cause_span = []
      for j in range(0,len(texts)):
        if j not in relevant_contexts_dict.keys():
          cause_span.append([0,0,0])
        else:
          span_text = relevant_contexts_dict[j]
          actual_text = texts[j]

          span_start = actual_text.find(span_text)
          span_end = span_start + len(span_text)

          cause_span.append([1,span_start,span_end])
      cause_spans = cause_spans + cause_span

  batch = {"emotion":emotion,"target_utterance":target_utterence, "context":conversation_history,
             "cause_spans":cause_spans}

  return batch






In [ ]:
import pickle
with open('validation_prediction_data', 'rb') as f:
    validation_prediction_data = pickle.load(f)

In [ ]:
validation_prediction_data.keys()

dict_keys(['val_batch_conv', 'val_batch_emo', 'val_batch_start', 'val_batch_end', 'val_start_preds', 'val_start_ends'])

In [ ]:
class EmotionClassifier(torch.nn.Module):
    def __init__(self, output_size: int, hidden_size: int, model_name='Nakul24/Spanbert-emotion-extraction'):
        super().__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size

        # Initialize BERT, which we use instead of a single embedding layer.
        self.bert = BertModel.from_pretrained(model_name)

        # TODO [OPTIONAL]: Updating all BERT parameters can be slow and memory intensive.
        # Freeze them if training is too slow. Notice that the learning
        # rate should probably be smaller in this case.
        # Uncommenting out the below 2 lines means only our classification layer will be updated.

        # for param in self.bert.parameters():
        #     param.requires_grad = False

        self.bert_hidden_dimension = self.bert.config.hidden_size

        # TODO: Add an extra hidden layer in the classifier, projecting
        #      from the BERT hidden dimension to hidden size. Hint: torch.nn.Linear()

        self.hidden_layer = torch.nn.Linear(self.bert_hidden_dimension, self.hidden_size)

        # TODO: Add a relu nonlinearity to be used in the forward method
        #      https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html

        self.relu = torch.nn.ReLU()

        self.classifier = torch.nn.Linear(self.hidden_size, self.output_size)
        self.log_softmax = torch.nn.LogSoftmax(dim=2)

#         self.span_start_classifier = torch.nn.Linear(self.bert_hidden_dimension, 1)
#         self.softmax_starts = torch.nn.Softmax(dim=1)

#         self.span_end_classifier = torch.nn.Linear(self.bert_hidden_dimension, 1)
#         self.softmax_ends = torch.nn.Softmax(dim=1)

    def encode_text(
        self,
        symbols: Dict
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols BERT.
            Then, get CLS represenation.

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: CLS token embedding
        """
        # First we get the contextualized embedding for each input symbol
        # We no longer need an LSTM, since BERT encodes context and
        # gives us a single vector describing the sequence in the form of the [CLS] token.
        encoded_sequence = self.bert(**symbols)
        # TODO: Get the [CLS] token
        #      The BertModel output. See here: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        #      and check the returns for the forward method.
        # We want to return a tensor of the form batch_size x 1 x bert_hidden_dimension
        # print(encoded_sequence.last_hidden_state[:, :1, :].shape)
        # Return only the first token's embedding from the last_hidden_state. Hint: using list slices
        return encoded_sequence.last_hidden_state[:, :1, :]

    def forward(
        self,
        symbols: Dict,
    ) -> torch.Tensor:
        """_summary_

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: _description_
        """
        encoded_sequence = self.bert(**symbols)

        encoded_sents = encoded_sequence.last_hidden_state[:, :1, :]
        # encoded_sents = self.encode_text(symbols)
        output = self.hidden_layer(encoded_sents)
        output = self.relu(output)
        output = self.classifier(output)

        # span_start_logits = self.span_start_classifier(encoded_sequence.last_hidden_state)
        # start_probs = self.softmax_starts(span_start_logits)
        # print (start_probs.shape)
        # span_end_logits = self.span_end_classifier(encoded_sequence.last_hidden_state)
        # end_probs = self.softmax_ends(span_end_logits)

        return self.log_softmax(output)#, start_probs, end_probs

In [ ]:
model.eval()

print("Evaluating dev...")
all_preds = []
all_labels = []
all_starts = []
all_ends = []
for sents, labels in tqdm(zip(val_batch_conv, val_batch_emo), total=len(val_batch_conv)):
    # pred = predict(model, sents).cpu()
    with torch.no_grad():
        pred = predict(trained_model, sents.to(device))
    all_preds.extend(pred)
    all_labels.extend(list(labels.cpu().numpy()))
    # all_starts.append(start_probs)
    # all_ends.append(end_probs)

dev_f1 = macro_f1(all_preds, all_labels, set(all_labels))
print(f"Dev F1 {dev_f1}")

# all_starts = np.concatenate([x.reshape(-1) for x in all_starts])
# dev_start_auc = roc_auc_score(dev_span_starts_all, all_starts)
# print(f"Dev START AUC: {train_start_auc}")

# all_ends = np.concatenate([x.reshape(-1) for x in all_ends])
# dev_end_auc = roc_auc_score(dev_span_ends_all, all_ends)
# print(f"Dev END AUC: {dev_end_auc}")


In [ ]:
# For making predictions at test time
def predict(model: torch.nn.Module, sents: torch.Tensor) -> List:
    logits = model(sents.to(device))
    return list(torch.argmax(logits, axis=2).squeeze().cpu().numpy())

In [ ]:
def decode_span(conversation):
  ids = [i['utterance_ID'] for i in conversation['conversation']]
  emotions = [i['emotion'] for i in conversation['conversation']]

  non_neutral_emotions = [i+1 for i in range(len(emotions)) if emotions[i] != 'neutral']

  texts = [i['text'] for i in conversation['conversation']]
  speakers = [i['speaker'] for i in conversation['conversation']]
  speaker_dict = {}
  ind = 1
  for speaker in speakers:
      if speaker not in speaker_dict.keys():
          if ind <= 7:
              speaker_dict[speaker] = '[SP'+str(ind)+']'
              ind = ind+1
          else:
              speaker_dict[speaker] = '[SP8]'
  speakers = [speaker_dict[speaker] for speaker in speakers]

  history = [speakers[i]+conversation['conversation'][i]['text'] for i in range(0,len(texts))]
  history = "[SEP]".join(history)

  ec_pairs = conversation['emotion-cause_pairs']

  ec_dict = {}
  for ind in non_neutral_emotions:
    ec_dict[ind] = []

  for ec in ec_pairs:
    ec_dict[int(ec[0].split('_')[0])].append(ec[1])

  emotion = []
  target_utterence = []
  evidence_utterence = []
  conversation_history = []
  cause_spans = []

  for i in range(0, len(texts)):
    if emotions[i] != 'neutral':
      emotion = emotion + ([emotions[i]]*len(texts))
      target_utterence = target_utterence + ([texts[i]]*len(texts))
      #evidence_utterence = evidence_utterence + texts

      conversation_history_single = [text + ' [SEP] '+ history for text in texts]

      conversation_history = conversation_history + (conversation_history_single)
      relevant_contexts = ec_dict[i+1]
      relevant_contexts_dict = {}
      for context in relevant_contexts:
        relevant_contexts_dict[int(context.split('_')[0])-1] = context.split('_')[1]
      cause_span = []
      for j in range(0,len(texts)):
        if j not in relevant_contexts_dict.keys():
          cause_span.append([0,0,0])
        else:
          span_text = relevant_contexts_dict[j]
          actual_text = texts[j]

          span_start = actual_text.find(span_text)
          span_end = span_start + len(span_text)

          cause_span.append([1,span_start,span_end])
      cause_spans = cause_spans + cause_span

  batch = {"emotion":emotion,"target_utterance":target_utterence, "context":conversation_history,
             "cause_spans":cause_spans}

  return batch

In [ ]:
# For making predictions at test time
def predict(model: torch.nn.Module, sents: torch.Tensor) -> List:
    logits = model(**sents)
    start_probs = logits.start_logits.softmax(dim=1)
    end_probs = logits.end_logits.softmax(dim=1)
    return start_probs.cpu().detach().numpy(), end_probs.cpu().detach().numpy()

In [ ]:
model.eval()

print("Evaluating dev...")
all_preds = []
all_labels = []
all_starts = []
all_ends = []
for sents, labels in tqdm(zip(val_batch_conv, val_batch_emo), total=len(val_batch_conv)):
    # pred = predict(model, sents).cpu()
    with torch.no_grad():
        start_probs, end_probs = predict(trained_model, sents.to(device))
    # all_preds.extend(pred)
    all_labels.extend(list(labels.cpu().numpy()))
    all_starts.append(start_probs)
    all_ends.append(end_probs)

# dev_f1 = macro_f1(all_preds, all_labels, set(all_labels))
# print(f"Dev F1 {dev_f1}")

# all_starts = np.concatenate([x.reshape(-1) for x in all_starts])
# dev_start_auc = roc_auc_score(dev_span_starts_all, all_starts)
# print(f"Dev START AUC: {train_start_auc}")

# all_ends = np.concatenate([x.reshape(-1) for x in all_ends])
# dev_end_auc = roc_auc_score(dev_span_ends_all, all_ends)
# print(f"Dev END AUC: {dev_end_auc}")


In [ ]:
val_start_preds = [s.argmax(axis=1) for s in all_starts]
val_end_preds = [s.argmax(axis=1) for s in all_ends]